In [65]:
# training loop - train_step
# testing loop - test loop

# Dataset
# setup traing data
from torchvision import datasets
import torchvision.transforms as transforms

# Define a transform to convert the image to a tensor
transform = transforms.Compose([
    transforms.ToTensor()
])

train_data = datasets.FashionMNIST(root = "data",
                      train=True,
                      download=True,
                      transform=transform)


test_data = datasets.FashionMNIST(root = "data",
                      train=False,
                      download=True,
                      transform=transform,)

In [66]:
from torch.utils.data.dataloader import DataLoader

train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_data_loader = DataLoader(test_data, shuffle=True, batch_size=32)

In [67]:
from torch import nn
import torch

class FashionMNISTModel(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.layer_stack(x)
    
    


torch.manual_seed(42)

model_1 = FashionMNISTModel(
    input_shape= 784,
    hidden_units=10,
    output_shape=len(train_data.classes)
    
).to("cpu")


from helper import accuracy_fn

loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model_1.parameters(), lr=0.1)

In [68]:
import torch
from torch import nn
from helper import accuracy_fn

# device agonostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

def train_step(model: nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn : nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn = accuracy_fn,
               device: torch.device = device):
    
    """Performs training steps of model with corresponding dataloader"""
    
    train_loss, train_acc = 0, 0
    
    model.train()

    
    for batch, (X,y) in enumerate(data_loader):
        
        X,y = X.to(device), y.to(device)
        
        train_logits = model(X)
        
        # In batch u should accumulate the train loss and acc
        loss = loss_fn(train_logits, y)
        train_loss += loss 
        train_acc  = accuracy_fn(y, train_logits.argmax(dim=1))
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        if batch % 400 == 0:
            print(f"Look at {batch * len(X)} / {len(train_data) } samples")
    
    train_loss = train_loss / len(train_data_loader)
    train_acc = train_acc / len(train_data_loader)
    
    print(f"Train loss : {train_loss} | Train Acc : {train_acc} ")
    
        
    

In [69]:
def test_step(model: torch.nn.Module,
              data_loader : torch.utils.data.DataLoader,
              loss_fn: nn.Module,
              accuracy_fn= accuracy_fn,
              device: torch.device = device):
    
    test_loss, test_acc = 0, 0
    
    model.eval()        

    with torch.inference_mode():

        for X,y in data_loader:
            
            X,y = X.to(device), y.to(device)

            test_logits = model(X)
            
            loss = loss_fn(test_logits, y)
            test_loss += loss
            
            test_acc += accuracy_fn(y, test_logits.argmax(dim=1))
        
        test_loss = test_loss / len(data_loader)
        test_acc = test_acc / len(data_loader)
        
        print(f"Test loss : {test_loss} |  Test acc : {test_acc}")
            
    

In [70]:
from timeit import default_timer as timer

start_time_cpu = timer()
EPOCHS =3

for epoch in range(EPOCHS):
    print(f"EPOCH check : {epoch}\n------")
    train_step(model=model_1, data_loader=train_data_loader, accuracy_fn=accuracy_fn, loss_fn=loss_fn,optimizer=optimizer)
    test_step(model=model_1, data_loader=train_data_loader, accuracy_fn=accuracy_fn, loss_fn=loss_fn)

end_time_cpu = timer()

print(f"Total time taken : {end_time_cpu - start_time_cpu}")


EPOCH check : 0
------
Look at 0 / 60000 samples
Look at 12800 / 60000 samples
Look at 25600 / 60000 samples
Look at 38400 / 60000 samples
Look at 51200 / 60000 samples
Train loss : 1.0587784051895142 | Train Acc : 0.04 
Test loss : 0.9633945226669312 |  Test acc : 65.22333333333333
EPOCH check : 1
------
Look at 0 / 60000 samples
Look at 12800 / 60000 samples
Look at 25600 / 60000 samples
Look at 38400 / 60000 samples
Look at 51200 / 60000 samples
Train loss : 0.9117913842201233 | Train Acc : 0.04 
Test loss : 0.9208199977874756 |  Test acc : 66.37833333333333
EPOCH check : 2
------
Look at 0 / 60000 samples
Look at 12800 / 60000 samples
Look at 25600 / 60000 samples
Look at 38400 / 60000 samples
Look at 51200 / 60000 samples
Train loss : 0.881722092628479 | Train Acc : 0.03666666666666667 
Test loss : 0.8477543592453003 |  Test acc : 68.45833333333333
Total time taken : 7.916713334000633


In [71]:
model_1

FashionMNISTModel(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): ReLU()
  )
)

In [76]:
import torch.utils.data.dataloader
from tqdm.auto import tqdm

def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.dataloader,
               loss : torch.nn.Module,
               accuracy_fn
              ):
    loss, acc = 0, 0
    model.eval()
    for X,y in tqdm(data_loader, colour="green"):
        test_logits = model(X)
        loss += loss_fn(test_logits, y)
        acc += accuracy_fn(y_true=y, y_pred=test_logits.argmax(dim=1))
    
    loss = loss / len(data_loader)
    acc = acc / len(data_loader)        
    
    return {"model_name" : model.__class__.__name__,
            "model_loss" : loss,
            "model_acc" : acc}

In [77]:
model_1_results = eval_model(model=model_1, data_loader=test_data_loader, loss=loss_fn, accuracy_fn=accuracy_fn)

  0%|          | 0/313 [00:00<?, ?it/s]

100%|██████████| 313/313 [00:00<00:00, 1279.81it/s]


In [78]:
model_1_results

{'model_name': 'FashionMNISTModel',
 'model_loss': tensor(0.8907, grad_fn=<DivBackward0>),
 'model_acc': 67.0726837060703}